# Text Emotion Detection

**importing essentials**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as fnc

**load dataset #1**

In [2]:
training_data1 = pd.read_excel("all_data/training_DS1.xlsx")
print(training_data1.groupby("reaction").apply(list))
print(f"shape = {training_data1.shape}")
print(f"dtype = {training_data1.dtypes}")
print(f"value count = \n{training_data1['reaction'].value_counts()}")

training_list = training_data1.values.tolist()
print("\n")
testing_data1 = pd.read_excel("all_data/testing_DS2.xlsx")
print(testing_data1.groupby("reaction").apply(list))
print(f"shape = {testing_data1.shape}")
print(f"dtype = {testing_data1.dtypes}")
print(f"value count = \n{testing_data1['reaction'].value_counts()}")

testing_list = training_data1.values.tolist()

reaction
angry      [text, reaction]
disgust    [text, reaction]
fear       [text, reaction]
happy      [text, reaction]
sad        [text, reaction]
dtype: object
shape = (3096, 2)
dtype = text        object
reaction    object
dtype: object
value count = 
happy      1086
sad         721
angry       720
disgust     357
fear        212
Name: reaction, dtype: int64


reaction
angry      [text, reaction]
disgust    [text, reaction]
fear       [text, reaction]
happy      [text, reaction]
sad        [text, reaction]
dtype: object
shape = (1265, 2)
dtype = text        object
reaction    object
dtype: object
value count = 
happy      311
disgust    289
sad        275
angry      208
fear       182
Name: reaction, dtype: int64


**load dataset #2**

In [3]:
training_data2 = pd.read_excel("all_data/training_DS2.xlsx")
print(training_data2.groupby("reaction").apply(list))
print(f"shape = {training_data2.shape}")
print(f"dtype = {training_data2.dtypes}")
print(f"value count = \n{training_data2['reaction'].value_counts()}")

training_list = training_data2.values.tolist()
print("\n")
testing_data2 = pd.read_excel("all_data/testing_DS1.xlsx")
print(testing_data2.groupby("reaction").apply(list))
print(f"shape = {testing_data2.shape}")
print(f"dtype = {testing_data2.dtypes}")
print(f"value count = \n{testing_data2['reaction'].value_counts()}")

testing_list = training_data2.values.tolist()

reaction
angry      [text, reaction]
disgust    [text, reaction]
fear       [text, reaction]
happy      [text, reaction]
sad        [text, reaction]
dtype: object
shape = (3720, 2)
dtype = text        object
reaction    object
dtype: object
value count = 
happy      883
sad        810
disgust    799
angry      629
fear       599
Name: reaction, dtype: int64


reaction
angry      [text, reaction]
disgust    [text, reaction]
fear       [text, reaction]
happy      [text, reaction]
sad        [text, reaction]
dtype: object
shape = (1032, 2)
dtype = text        object
reaction    object
dtype: object
value count = 
happy      351
angry      269
sad        224
disgust    112
fear        76
Name: reaction, dtype: int64


**data setting**

In [4]:
def clean_text(text):
    text = str(text)
    print(text)
    if "," in text:
        return str(text.replace(",",""))
    if "." in text:
        return str(text.replace(".",""))
    if "।" in text:
        return str(text.replace("।",""))
    if "|" in text:
        return str(text.replace("|",""))
    return str(text)
#     return "".join(char for char in text if char not in punchuation)
    

**emotion scoring**

In [5]:
def get_emotion(emotion):
    if emotion == "happy":
        return 0
    elif emotion == "sad":
        return 1
    elif emotion == "angry":
        return 2
    elif emotion == "disgust":
        return 3
    elif emotion == "fear":
        return 4

**ds1**

In [6]:
print(training_data1)
print("\n")
print(testing_data1)

                                                   text reaction
0     অপরাদি যে হোক, যে কোন বয়সের হোক না কেন, বিচারে...    happy
1     হাইরে মূর্খ বাঙালি। ডিজিটাল শব্দের অর্থ না বুঝ...     fear
2               নিশ্চিন্তে ভেঙে যাও, কেউ বলার নেই......     fear
3                        কেয়ামতের মাঠে তোমাদের ছাড়া নট।     fear
4     জনাব ইউসুফ এর মত কেউ অসহায় মানুষদের পাশে থাকলে...    happy
...                                                 ...      ...
3091                                            ধন্যবাদ    happy
3092  এগুলো মানব রচিত , বেদায়াত । ১লা বৈশাখ থেকে মুস...     fear
3093  আপনি সঠিক ও সৎ পথে রয়েছেন, বিজয় সব সময় সততার হ...    happy
3094                                 স্যালুট ইউ স্যার "    happy
3095                                               রাইট    happy

[3096 rows x 2 columns]


                                                   text reaction
0     রিমঝিম এই বৃষ্টিতে  ঈদ কাটাবো সৃষ্টিতে   খুশির...    happy
1     এরই মধ্যে আবার মেয়র সাহেব বললেন  ডেঙ্গুতে মানু...  disgus

**ds2**

In [7]:
print(training_data2)
print("\n")
print(testing_data2)

                                                   text reaction
0     মামা ভয়ে ভয়ে লুকিং গ্লাসের কাপড় সরিয়ে দিয়...     fear
1     এই বয়সে নাতি নাতনিদের সঙ্গে আনন্দে মেতে থাকার ...  disgust
2                         আপু আপনাকে খুব সুন্দর লাগছে      happy
3     তোমার শহরের কোণে কেউ মায়া জমায় কি  আমি এখনো ভা...      sad
4     এর মাথা পুরোটাই নষ্ট  পশ্চিম বঙ্গবাসী একে ভোট ...    angry
...                                                 ...      ...
3715  আমি কষ্টের স্মৃতি নিয়ে করি বসবাস আমায় নিয়ে কখন...      sad
3716  আজ থেকে অফিসে এবং বাসায় প্রথম আলো পত্রিকা  বাদ...    angry
3717  এই নোয়াখাইল্লা বাইনচুদেরে বেধে লেংটা করে রোদে ...    angry
3718  হিরু আলম জায়েদ খান  ওরা ও  নায়েক আর নোবেল ও এক...  disgust
3719  সত্যি সাকিব ভাই আজ আপনি আবার প্রমান করলেন কেন ...    happy

[3720 rows x 2 columns]


                                                   text reaction
0     কোরিয়ার ইনছন এয়ারপোর্টের মত শাহজালাল এয়ারপোর্ট...    happy
1                                           বাহ!সুন্দর।    happ

**cleaning ds1**

In [8]:
training_data1["text"] = training_data1["text"].apply(clean_text)
training_data1["label"] = training_data1["reaction"].apply(get_emotion)
training_data1["num_of_words"] = training_data1["text"].apply(lambda x:len(str(x).split()))

print("\n split \n")

testing_data1["text"] = testing_data1["text"].apply(clean_text)
testing_data1["label"] = testing_data1["reaction"].apply(get_emotion)
testing_data1["num_of_words"] = testing_data1["text"].apply(lambda x:len(str(x).split()))

অপরাদি যে হোক, যে কোন বয়সের হোক না কেন, বিচারের মুখা মুখি হতে হবে,,,
হাইরে মূর্খ বাঙালি। ডিজিটাল শব্দের অর্থ না বুঝিয়া আর কতো এ শব্দের অপব্যাবহার দেখবো?
নিশ্চিন্তে ভেঙে যাও, কেউ বলার নেই......
কেয়ামতের মাঠে তোমাদের ছাড়া নট।
জনাব ইউসুফ এর মত কেউ অসহায় মানুষদের পাশে থাকলে দেশের অবস্থার উন্নতি হতে ১ বছর ও লাগবে না......!!!!...
বাংলাদেশে মুসলমান যে দেশে যাবে বা দুকবে সে দেশ নয়রাজ্যগর্তা শিষ্ঠি হয় এশিয়ার মধ্যো প্রায্য দেশ মুসলিম দেশ সে দেশে মুসলিম রা বাংলাদেশের মুসলিম কে দেখতে পারে না । উরা বলে বাংলা নপর হাঁরামী মালায়সিয়া লুকেরা বলে বাংলা ওয়ারাং আলিবাবা লুকচা ওয়ারাং তাবাগুড আর বাংলাদেশ মুসলিম ম্রাংমারে গেলে সব সময় গন্দগোল খুন খারাপ লেগেয় তাকবে কারণ মায়ানমারা চুরি লুচা মিথ্যা বাদী কে প্রশ্রয় দেয়না পড়ে দুই দেশে বন্ধুত্ব রাতারাতি নষ্ঠহবে
2nd time in BANGLADESH. .😊 অভিনন্দন মিঃ বেনজির ।
ঠিক আছে
আমার মনে হয় ইলেক্ট্রনিক এবং প্রিন্ট মিডিয়াদের সাথে আপনাদের সার্বক্ষণিক একটা লিয়াজু রাখা উচিত। সমন্নিত উদ্যোগ ছাড়া বেশি দূর যাওয়া কস্ট হবে। ভেতরের খবর গুলো যত বেশি মানুষ জানবে অপরাধিরা তত বেশি ভয়ে থাকবে

In [9]:
print(training_data1)
print("\n split\n")
print(training_data1)

                                                   text reaction  label  \
0     অপরাদি যে হোক যে কোন বয়সের হোক না কেন বিচারের ...    happy      0   
1     হাইরে মূর্খ বাঙালি ডিজিটাল শব্দের অর্থ না বুঝি...     fear      4   
2                নিশ্চিন্তে ভেঙে যাও কেউ বলার নেই......     fear      4   
3                         কেয়ামতের মাঠে তোমাদের ছাড়া নট     fear      4   
4     জনাব ইউসুফ এর মত কেউ অসহায় মানুষদের পাশে থাকলে...    happy      0   
...                                                 ...      ...    ...   
3091                                            ধন্যবাদ    happy      0   
3092  এগুলো মানব রচিত  বেদায়াত । ১লা বৈশাখ থেকে মুসল...     fear      4   
3093  আপনি সঠিক ও সৎ পথে রয়েছেন বিজয় সব সময় সততার হয়...    happy      0   
3094                                 স্যালুট ইউ স্যার "    happy      0   
3095                                               রাইট    happy      0   

      num_of_words  
0               14  
1               14  
2                6  
3              

In [10]:
print(f"training dataset 1 = \n{training_data1['reaction'].value_counts()}")
print(f"testing dataset 1 = \n{testing_data1['reaction'].value_counts()}")

training dataset 1 = 
happy      1086
sad         721
angry       720
disgust     357
fear        212
Name: reaction, dtype: int64
testing dataset 1 = 
happy      311
disgust    289
sad        275
angry      208
fear       182
Name: reaction, dtype: int64


**cleaning ds2**

In [11]:
training_data2["text"] = training_data2["text"].apply(clean_text)
training_data2["label"] = training_data2["reaction"].apply(get_emotion)
training_data2["num_of_words"] = training_data2["text"].apply(lambda x:len(str(x).split()))

print("\n split \n")

testing_data2["text"] = testing_data2["text"].apply(clean_text)
testing_data2["label"] = testing_data2["reaction"].apply(get_emotion)
testing_data2["num_of_words"] = testing_data2["text"].apply(lambda x:len(str(x).split()))

মামা ভয়ে ভয়ে লুকিং গ্লাসের কাপড় সরিয়ে দিয়ে দেখে বাচ্চাটি একটা রক্তাক্ত পা চিবাই চিবাই খাচ্ছে  
এই বয়সে নাতি নাতনিদের সঙ্গে আনন্দে মেতে থাকার কথা তা নয় এ বুড়ো বয়সের ভীমরতি
আপু আপনাকে খুব সুন্দর লাগছে  
তোমার শহরের কোণে কেউ মায়া জমায় কি  আমি এখনো ভাবি সেই তোমাকে ফেরানো যাবে কি 
এর মাথা পুরোটাই নষ্ট  পশ্চিম বঙ্গবাসী একে ভোট দেয়   ছ্যা ছ্যা ছ্যা
আমি গত সপ্তাহের কতটা মনে পড়ছি তা দেখে আমি চমকে উঠতে শুরু করি আমি নিশ্চিত অসুস্থ কিছুটা নার্ভাস বোধ করি
 শেষ সময়ে এসে কিছু তিক্ত ঘটনা ঘটে গেলো কিন্তু এর জের ধরে নিজ নিজ ইগো নিয়ে যদি সবাই বসে থাকি তাহলে আজ থেকে অনেক বছর পর অবশ্যই পোড়াবে 
রবি আজিয়াটা লিমিটেডের দুই ব্র্যান্ড রবি ও এয়ারটেলের আয়োজনে বাংলাদেশে প্রথমবারের মতো অনুষ্ঠিত হলো আন্তর্জাতিক অনলাইন গেমিং প্রতিযোগিতা  আজিয়াটা গেম হিরো   প্রতি দলে  জন করে  দলের  জন প্রতিযোগী বাংলাদেশ পর্বের চূড়ান্ত পর্বে অংশগ্রহণ করেন  অংশগ্রহণকারী টি দল প্রায়  লাখ টাকার পুরস্কার জিতেছে 
একটা সেকেন্ডও স্কিপ করিনি পুরাই আগুন ছিলো লাভ ফ্রম বাংলাদেশ 
জাহালমের আটক থাকার ঘটনাকে ন্যাক্কারজনক বলে সমালোচনা করে বিএনপির

In [12]:
print(training_data2)
print("\n split\n")
print(training_data2)

                                                   text reaction  label  \
0     মামা ভয়ে ভয়ে লুকিং গ্লাসের কাপড় সরিয়ে দিয়...     fear      4   
1     এই বয়সে নাতি নাতনিদের সঙ্গে আনন্দে মেতে থাকার ...  disgust      3   
2                         আপু আপনাকে খুব সুন্দর লাগছে      happy      0   
3     তোমার শহরের কোণে কেউ মায়া জমায় কি  আমি এখনো ভা...      sad      1   
4     এর মাথা পুরোটাই নষ্ট  পশ্চিম বঙ্গবাসী একে ভোট ...    angry      2   
...                                                 ...      ...    ...   
3715  আমি কষ্টের স্মৃতি নিয়ে করি বসবাস আমায় নিয়ে কখন...      sad      1   
3716  আজ থেকে অফিসে এবং বাসায় প্রথম আলো পত্রিকা  বাদ...    angry      2   
3717  এই নোয়াখাইল্লা বাইনচুদেরে বেধে লেংটা করে রোদে ...    angry      2   
3718  হিরু আলম জায়েদ খান  ওরা ও  নায়েক আর নোবেল ও এক...  disgust      3   
3719  সত্যি সাকিব ভাই আজ আপনি আবার প্রমান করলেন কেন ...    happy      0   

      num_of_words  
0               16  
1               15  
2                5  
3              

In [13]:
print(f"training dataset 2 = \n{training_data2['reaction'].value_counts()}")
print(f"testing dataset 2 = \n{testing_data2['reaction'].value_counts()}")

training dataset 2 = 
happy      883
sad        810
disgust    799
angry      629
fear       599
Name: reaction, dtype: int64
testing dataset 2 = 
happy      351
angry      269
sad        224
disgust    112
fear        76
Name: reaction, dtype: int64


**debug**

In [14]:
# debug
max_text_len = training_data1["num_of_words"].max()
print(max_text_len)
x = training_data1["text"]
y = 0
for i in x:
    if i == "":
        y+=1
print(y)
print(x)

522
0
0       অপরাদি যে হোক যে কোন বয়সের হোক না কেন বিচারের ...
1       হাইরে মূর্খ বাঙালি ডিজিটাল শব্দের অর্থ না বুঝি...
2                  নিশ্চিন্তে ভেঙে যাও কেউ বলার নেই......
3                           কেয়ামতের মাঠে তোমাদের ছাড়া নট
4       জনাব ইউসুফ এর মত কেউ অসহায় মানুষদের পাশে থাকলে...
                              ...                        
3091                                              ধন্যবাদ
3092    এগুলো মানব রচিত  বেদায়াত । ১লা বৈশাখ থেকে মুসল...
3093    আপনি সঠিক ও সৎ পথে রয়েছেন বিজয় সব সময় সততার হয়...
3094                                   স্যালুট ইউ স্যার "
3095                                                 রাইট
Name: text, Length: 3096, dtype: object


**spliting ds1**

In [20]:

X_train, X_test, Y_train, Y_test = train_test_split(training_data1["text"].tolist(),\
                                                   training_data1["label"].tolist(),\
                                                   test_size = 0.2,\
                                                   stratify = training_data1["label"].tolist(),\
                                                   random_state = 0)
print(f"Xtrain = {len(X_train)}")
print(f"Xtest = {len(X_test)}")
print(f"Ytrain = {len(Y_train)}")
print(f"Ytest = {len(Y_test)}")

train_set = training_data1
test_set = testing_data1

Xtrain = 2476
Xtest = 620
Ytrain = 2476
Ytest = 620


Torch

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

False


In [24]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
def yield_tokens(train_set):
    for text in train_set:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_set), specials=[""])
vocab.set_default_index(vocab[""])
print(len(vocab.get_stoi()))
print(vocab.get_stoi())

5
{'label': 1, '': 0, 'num_of_words': 2, 'reaction': 3, 'text': 4}


In [25]:
text_pipeline = lambda x:vocab(tokenizer(x))
label_pipeline = lambda x:int(x)

In [26]:
text_pipeline("চরম চাটাঁ লাথি চাই টুপি")

[0, 0, 0, 0, 0]

In [27]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
        
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [28]:
num_training_data = len(train_set)
num_classes = len(training_data1['reaction'].value_counts())
vocab_size = len(vocab)
embeding_size = int(min(500,vocab_size/2))

print(f"training_Data = {num_training_data}")
print(f"classes = {num_classes}")
print(f"vocab = {vocab_size}")
print(f"embeding = {embeding_size}")

training_Data = 3096
classes = 5
vocab = 5
embeding = 2


Model

**TextClassificationModel #1**

In [29]:
#fc1, fc2, fc3
class TextClassificationModel1(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel1, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,100)
        self.fc2 = nn.Linear(100,50)
        self.fc3 = nn.Linear(50, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = fnc.relu(self.fc1(embedded))
        x = fnc.relu(self.fc2(x))
        x = self.fc3(x)
        return x

**TextClassificationModel #2**

In [30]:
#fc1, fc2, fc3, fc4, fc5
class TextClassificationModel2(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel2, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,250)
        self.fc2 = nn.Linear(250,170)
        self.fc3 = nn.Linear(170,85)
        self.fc4 = nn.Linear(85,40)
        self.fc5 = nn.Linear(40, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
        self.fc5.weight.data.uniform_(-initrange, initrange)
        self.fc5.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = fnc.relu(self.fc1(embedded))
        x = fnc.relu(self.fc2(x))
        x = fnc.relu(self.fc3(x))
        x = fnc.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [31]:
model1 = TextClassificationModel1(vocab_size, embeding_size, num_classes).to(device)
model2 = TextClassificationModel2(vocab_size, embeding_size, num_classes).to(device)

**Train Test 1**

In [32]:
import time

def train1(dataloader):
    model1.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model1(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def test1(dataloader):
    model1.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model1(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

**Train Test 2**

In [33]:
import time

def train2(dataloader):
    model2.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model2(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def test2(dataloader):
    model2.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model2(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

**run on train test #1 model #1**

In [34]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 2  # learning rate
BATCH_SIZE = 150 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_set
print(type(train_set))
# test_iter2 =test_dat 
test_iter2 = test_set



train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
print(type(train_dataloader))
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
#                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train1(train_dataloader)
    accu_test = test1(test_dataloader)
    if total_accu is not None and total_accu > accu_test:
        scheduler.step()
    else:
        total_accu = accu_test
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_test))
    print('-' * 59)
    print(f"total accu = {total_accu}")

<class 'pandas.core.frame.DataFrame'>
<class 'torch.utils.data.dataloader.DataLoader'>


KeyError: 1316

**run on train test #2 model #2**

In [35]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 2  # learning rate
BATCH_SIZE = 150 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model2.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_set
print(type(train_set))
# test_iter2 =test_dat 
test_iter2 = test_set



train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
print(type(train_dataloader))
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
# test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
#                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train2(train_dataloader)
    accu_test = test2(test_dataloader)
    if total_accu is not None and total_accu > accu_test:
        scheduler.step()
    else:
        total_accu = accu_test
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_test))
    print('-' * 59)
    print(f"total accu = {total_accu}")

<class 'pandas.core.frame.DataFrame'>
<class 'torch.utils.data.dataloader.DataLoader'>


KeyError: 2940